In [1]:
# step1. 라이브러리 불러오기
import os
import json
from datetime import date
from typing import Dict, List, Any
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv

load_dotenv()
print("✅ 라이브러리 불러오기 완료!")

✅ 라이브러리 불러오기 완료!


In [2]:
# step2. LLM 초기화 및 설정
try:
    # 최종 보고서 작성은 품질이 중요하므로 GPT-4o-mini를 사용하되, 출력이 길어질 수 있음
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1) 
    print("✅ ChatOpenAI LLM 초기화 완료!")
except Exception as e:
    llm = None

✅ ChatOpenAI LLM 초기화 완료!


In [3]:
# step3. 보고서 생성 프롬프트 정의 (요청된 7개 섹션 구성 반영)
REPORT_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", 
         "당신은 AI 윤리 진단 보고서를 작성하는 전문가입니다. "
         "제공된 모든 정보를 바탕으로, 다음 목차를 엄격히 준수하여 **하나의 깔끔한 Markdown 형식의 보고서**를 한국어로 작성하세요. "
         "어떠한 추가 설명이나 텍스트 없이 순수 Markdown 텍스트만을 반환해야 합니다."
         "\n\n[보고서 목차]:"
         "\n1. EXECUTIVE SUMMARY (요약)"
         "\n2. SERVICE PROFILE (서비스 프로파일)"
         "\n3. RISK ASSESSMENT (리스크 평가)"
         "\n4. MITIGATION RECOMMENDATIONS (개선 권고)"
         "\n5. CONCLUSION (결론)"
         "\n6. REFERENCE (참고 자료)"
         "\n7. APPENDIX (부록)"
        ),
        ("human", 
         "--- 최종 보고서 작성을 위한 데이터 ---"
         "\n분석 날짜: {today_date}"
         "\n\n[1. 서비스 프로파일 정보]:\n{service_profile}"
         "\n\n[2. 리스크 평가 결과 (등급 및 요약)]:\n{assessment_result}"
         "\n\n[3. 개선 권고안 (실행 방안 포함)]:\n{recommendation_result}"
         "\n\n[4. 수집된 증거 (출처 및 요약)]:\n{all_evidences}"
         "\n\n위 데이터를 기반으로, 목차에 맞춰 **AI Ethics Risk Assessment Report**를 작성하세요."
        )
    ]
)

In [4]:
# step4. 보고서 생성 함수 정의 (State를 입력으로 받도록 변경)
def compose_report_and_update_state(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    State의 모든 정보를 취합하여 최종 Markdown 보고서를 생성하고 State에 저장합니다.
    """
    if not llm:
        state['report_status'] = "Error: LLM failed to initialize."
        return state
    
    # State에서 필요한 데이터 읽기
    service_profile = state.get('service_profile', {})
    assessment_result = state.get('assessment_result', {})
    recommendation_result = state.get('recommendation_result', [])
    evidence_data = state.get('evidence_data', {})
    
    if not all([service_profile, assessment_result, recommendation_result, evidence_data]):
        print("❌ State에 필수 데이터(Profile, Evidence, Assessment, Recommendation)가 부족합니다.")
        state['report_status'] = "Error: Missing upstream data."
        return state

    print(f"\n📝 Report Composer 시작: 최종 보고서 생성")
    
    # 보고서에 포함할 증거 정보 정리 (출처와 요약을 구분하여 전달)
    all_evidences = {
        'baseline_sources': [
            {'category': s['category'], 'source': s['source'], 'summary': s['summary']}
            for s in evidence_data.get('baseline_sources', [])
        ],
        'issue_sources': [
            {'category': s['category'], 'source': s['source'], 'summary': s['summary']}
            for s in evidence_data.get('issue_sources', [])
        ]
    }
    
    # 프롬프트 구성
    chain = REPORT_PROMPT | llm
    
    try:
        final_report_markdown = chain.invoke({
            "today_date": date.today().isoformat(),
            # JSON.dumps를 사용하여 LLM에게 구조화된 데이터를 명확히 전달
            "service_profile": json.dumps(service_profile, indent=2, ensure_ascii=False),
            "assessment_result": json.dumps(assessment_result, indent=2, ensure_ascii=False),
            "recommendation_result": json.dumps(recommendation_result, indent=2, ensure_ascii=False),
            "all_evidences": json.dumps(all_evidences, indent=2, ensure_ascii=False)
        }).content
        
    except Exception as e:
        final_report_markdown = f"## ❌ 보고서 생성 실패\n\nLLM 호출 중 오류 발생: {e}"
        state['report_status'] = "Error generating report."
        return state

    # 💡 State에 최종 보고서 저장
    state['final_report_markdown'] = final_report_markdown
    state['report_status'] = "Success"
    
    print("\n✅ Report Composer 완료 및 State 업데이트 완료!")
    return state

In [5]:
# step5. 테스트 실행 (전체 State 데이터의 더미 데이터 사용)

# Note: 이 테스트 State는 모든 에이전트의 출력을 시뮬레이션합니다.
final_test_state = {
    # 1. Service Profiler 결과 (Report Composer의 SERVICE PROFILE 섹션에 사용)
    'service_profile': {
        'service_name': '이력서 분석 추천 시스템', 
        'service_type': '채용 시스템 (고위험 분류)',
        'data_handling_method': '사용자 입력 이력서 및 직무 데이터 기반 분석, 개인 식별 정보는 익명화 후 사용',
        'user_impact_scope': '채용 결정에 직접적인 영향을 미침, 구직자의 기회 박탈 위험 내포',
        'diagnosed_risk_categories': ['bias', 'privacy', 'transparency']
    },
    # 2. Evidence Collector 결과 (Report Composer의 RISK ASSESSMENT 및 REFERENCE 섹션에 사용)
    'evidence_data': {
        'query': '이력서 분석 추천 시스템',
        'scores': {'bias': 0.8, 'privacy': 0.6, 'transparency': 0.4}, # 임의의 점수 추가
        'baseline_sources': [
            {'category': 'bias', 'source': 'EU AI Act', 'summary': '채용 AI는 고위험으로 분류되어 엄격한 공정성 기준 준수 의무가 있다.'},
            {'category': 'privacy', 'source': 'OECD AI 원칙', 'summary': '민감 정보 처리 시 데이터 주체의 통제권 보장 및 익명화가 중요하다.'}
        ], 
        'issue_sources': [
            {'category': 'bias', 'source': '2024년 5월 IT뉴스', 'summary': '최근 대기업 채용 AI에서 특정 성별에 대한 암묵적 차별 문제가 제기되었다.'}
        ]
    },
    # 3. Risk Assessor 결과 (Report Composer의 RISK ASSESSMENT 섹션에 사용)
    'assessment_result': {
        'service_name': '이력서 분석 추천 시스템',
        'assessed_risks': [
            {'category': 'bias', 'risk_level': 'High', 'assessment_summary': '채용 과정에 직접 영향, 사회적 이슈가 명확하여 즉각적인 공정성 감사가 필요함.', 'recommendation_focus': '데이터 다양성 확보 및 모델 공정성 감사'},
            {'category': 'privacy', 'risk_level': 'Limited', 'assessment_summary': '익명화 기술을 사용하지만, 재식별 가능성 잔존으로 지속적인 모니터링이 요구됨.', 'recommendation_focus': '익명화 수준 강화 및 동의 절차 명확화'},
            {'category': 'transparency', 'risk_level': 'Minimal', 'assessment_summary': '설명 의무는 있으나, 큰 이슈 없음. 문서화로 충분함.', 'recommendation_focus': '결정 근거 문서화'}
        ]
    },
    # 4. Mitigation Recommender 결과 (Report Composer의 MITIGATION RECOMMENDATIONS 섹션에 사용)
    'recommendation_result': [
        {'recommendation_title': '데이터 공정성 감사 의무화', 'mitigation_step': '정기적으로 학습 데이터의 성별, 학력, 지역별 분포를 분석하고, 편향성 완화 기술을 적용한다.', 'priority': 'High', 'effort_level': 'High', 'risk_category': 'bias', 'relevant_standard': 'EU AI Act'},
        {'recommendation_title': '익명화 수준 T-3 기법 도입', 'mitigation_step': 'K-익명성 기준을 강화하고, 익명화된 데이터셋에 대한 재식별 가능성 검증 프로세스를 도입한다.', 'priority': 'Medium', 'effort_level': 'Medium', 'risk_category': 'privacy', 'relevant_standard': 'OECD AI 원칙'}
    ]
}

print("\n" + "="*60)
print("📝 Report Composer 시작 (State 기반 테스트)...")
print("="*60)

# 최종 보고서 생성 및 State 업데이트
updated_state = compose_report_and_update_state(final_test_state)

# 최종 보고서 파일 생성 (사용자에게 전달되는 최종 결과물)
# 보고서 생성을 위해 File Generation Workflow를 따릅니다.
report_title = f"AI 윤리 리스크 진단 보고서 - {final_test_state['service_profile']['service_name']}"
report_content = updated_state.get('final_report_markdown', '보고서 생성 실패')

if updated_state.get('report_status') == 'Success':
    print("\n✅ 최종 보고서 파일이 생성되었습니다.")
    
    # File Generation Workflow 시작
    print(f"\nReport Composer가 최종 보고서 파일을 생성합니다.")
    
    print(f"보고서 제목: {report_title}")
    
    # 최종 결과물인 Markdown 보고서 파일을 생성합니다.
    print(updated_state.get('final_report_markdown', ''))
    
else:
    print(f"\n❌ 보고서 생성에 오류가 발생했습니다: {updated_state.get('report_status')}")


📝 Report Composer 시작 (State 기반 테스트)...

📝 Report Composer 시작: 최종 보고서 생성

✅ Report Composer 완료 및 State 업데이트 완료!

✅ 최종 보고서 파일이 생성되었습니다.

Report Composer가 최종 보고서 파일을 생성합니다.
보고서 제목: AI 윤리 리스크 진단 보고서 - 이력서 분석 추천 시스템
# AI Ethics Risk Assessment Report

## 1. EXECUTIVE SUMMARY (요약)
이 보고서는 "이력서 분석 추천 시스템"에 대한 AI 윤리 리스크 평가를 다룹니다. 본 시스템은 채용 과정에서 구직자에게 직접적인 영향을 미치며, 편향성, 개인정보 보호, 투명성의 세 가지 주요 리스크 카테고리를 진단하였습니다. 각 리스크에 대한 평가와 개선 권고안을 제시하여 시스템의 윤리적 운영을 보장하고자 합니다.

## 2. SERVICE PROFILE (서비스 프로파일)
- **서비스 이름**: 이력서 분석 추천 시스템
- **서비스 유형**: 채용 시스템 (고위험 분류)
- **데이터 처리 방법**: 사용자 입력 이력서 및 직무 데이터 기반 분석, 개인 식별 정보는 익명화 후 사용
- **사용자 영향 범위**: 채용 결정에 직접적인 영향을 미침, 구직자의 기회 박탈 위험 내포
- **진단된 리스크 카테고리**: bias, privacy, transparency

## 3. RISK ASSESSMENT (리스크 평가)
- **서비스 이름**: 이력서 분석 추천 시스템
- **평가된 리스크**:
  1. **편향성 (bias)**
     - **리스크 수준**: High
     - **평가 요약**: 채용 과정에 직접 영향, 사회적 이슈가 명확하여 즉각적인 공정성 감사가 필요함.
     - **권고 초점**: 데이터 다양성 확보 및 모델 공정성 감사
  2. **개인정보 보호 (privacy)**
     - **리스크 수준**: Limited
     - **평